In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!pip install edgartools -qq
!pip install ollama -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.6 MB/s eta 0:00:00


In [4]:
import duckdb

import pandas as pd
from edgar import *
from datetime import datetime, date
set_identity('nantyinminaye@gmail.com')
import subprocess
import ollama

In [5]:
!sudo apt update > /dev/null 2>&1
!sudo apt install pciutils lshw > /dev/null 2>&1
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null 2>&1

In [6]:
import subprocess
import ollama
import os
import time

# Set environment variable to show only errors
env = os.environ.copy()
env['OLLAMA_DEBUG'] = 'ERROR'

# Start Ollama server in background
process = subprocess.Popen(
    "ollama serve",
    shell=True,
    env=env,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# Give the server a moment to start
time.sleep(2)

# Pull the model (run this in a separate process)
pull_process = subprocess.run(
    ["ollama", "pull", "gemma3:12b"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
    env=env
)

if pull_process.returncode == 0:
    print("Model pulled successfully")
else:
    print(f"Error pulling model: {pull_process.stderr}")

Model pulled successfully


In [7]:
con = duckdb.connect("/content/drive/MyDrive/MFM project/bankruptcy_data.db")
bankrupted = con.execute("SELECT * FROM non_bankrupted").fetch_df().sort_values("non_b_ticker", ascending = False)
bankrupted['startdate'] = pd.to_datetime(bankrupted['startdate'], format = '%Y-%m-%d') + pd.offsets.DateOffset(years=-1)
bankrupted['startdate'] = bankrupted.startdate.astype(str)
# bankrupted = bankrupted[~bankrupted.Ticker.isin(['financials', 'financial2', 'bankrupted'])]

focused_tickers = []

for non_b_ticker, startdate in bankrupted[['non_b_ticker', 'startdate']].sort_values('non_b_ticker').values:
    company = Company(non_b_ticker)
    filings = company.get_filings(form=['10-Q', '10-K'])
    if (len(filings) >= 12):
        focused_tickers.append(non_b_ticker)

bankrupted = bankrupted[bankrupted.non_b_ticker.isin(focused_tickers)]
bankrupted.head()

,ticker,non_b_ticker,company_name,startdate,enddate
107,FALC,ZS,"Zscaler, Inc.",2018-07-06,2023-07-06
16,SYNE,XOM,Exxon Mobil Corporation,2016-08-27,2021-08-27
137,BIGGQ,WMT,Walmart Inc,2019-09-09,2024-09-09
6,YUMAQ,WMB,"Williams Companies, Inc.",2015-04-16,2020-04-16
121,BSFC,WFC,"The Chefs' Warehouse, Inc.",2019-12-19,2024-12-19


In [10]:
completed = pd.read_csv("/content/drive/MyDrive/MFM project/completed.txt", delimiter = "|", header = None)
completed.columns = ['ticker', 'report_date', 'shares', 'short_term_debt', 'long_term_debt', 'cash', 'notes']
completed.head()

,ticker,report_date,shares,short_term_debt,long_term_debt,cash,notes
0,ZS,2025-04-30,155696000.0,3.168399e+09,0.000000e+00,1.990890e+09,Shares outstanding is based on the 'Balance as...
1,ZS,2025-01-31,154725000.0,3.048866e+09,0.000000e+00,1.758506e+09,Shares outstanding are derived from the 'Balan...
2,ZS,2024-10-31,153415000.0,2.963610e+09,0.000000e+00,2.575419e+09,Shares outstanding is derived from the 'Balanc...
3,ZS,2024-07-31,152490000.0,3.112887e+09,0.000000e+00,1.423080e+09,"Shares outstanding are reported as 152,490,000..."
4,ZS,2024-04-30,151304000.0,1.701321e+09,1.137687e+09,1.259197e+09,"Shares outstanding are reported as 151,304,000..."


In [ ]:
import time
import re
import json
from datetime import datetime

process = subprocess.Popen(
    "ollama serve",
    shell=True,
    env=env,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

def get_prompt(report, period_of_report, ticker):
    return f"""
        You are a Chartered Financial Analyst with expertise in financial statement analysis.

        Analyze the following financial report data for ticker [{ticker}] as of [{period_of_report}].

        Financial Data:
        {report}

        Extract the following information with precision:

        1. **Outstanding Shares**: Find the total number of shares outstanding (not authorized or issued). Look for:
           - "Shares outstanding"
           - "Common shares outstanding"
           - "Weighted average shares outstanding"
           - Report the most recent figure

        2. **Short-term Debt**: Debt or liabilities due within 12 months for the most recent figure, including:
           - Current portion of long-term debt
           - Short-term borrowings
           - Notes payable (current)
           - Current liabilities

        3. **Long-term Debt**: Debt or liabilities due beyond 12 months for the most recent figure, including:
           - Long-term debt (excluding current portion)
           - Long-term notes payable
           - Bonds payable
           - Long-term borrowings
           - If there is no liabilities or debt due beyond 12 month, calculate it as: Total debt (or liabilities) - Short-term debt (liabilities)

        4. **Cash and Cash Equivalents**:
            - cash
            - liquidable assets

        CRITICAL REQUIREMENTS:
        - Do not estimate or assume values
        - Pay attention to scale indicators (thousands, millions, billions)
        - APPLY the scale factor to convert all values to actual amounts
        - If values are in thousands, multiply by 1,000
        - If values are in millions, multiply by 1,000,000
        - If values are in billions, multiply by 1,000,000,000
        - Use exact numbers from the most recent period shown
        - If using calculation method, note this in the notes field
        - Return the final scaled values, not the original values with scale factors

        Return ONLY a JSON object in this exact format:
        {{
            "ticker": "{ticker}",
            "date": "{period_of_report}",
            "shares": [final_scaled_number],
            "short_term_debt": [final_scaled_number],
            "long_term_debt": [final_scaled_number],
            "cash_equivalent": [final_scaled_number],
            "notes": "[any important notes about the data, calculation and scale applied]"
        }}
    """

def get_result(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            time.sleep(2)
            response = ollama.chat(
                model='gemma3:12b',
                messages=[{'role': 'user', 'content': prompt}],
                think=False,
                options={
                    'temperature': 0,
                    'top_p': 0.1,
                    'repeat_penalty': 1.1,
                    'num_ctx': 128000
                }
            )
            return response['message']['content']
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == max_retries - 1:
                return f"ERROR: Failed after {max_retries} attempts: {e}"
            time.sleep(5)

def remove_duplicates_from_string(text):
    """
    Remove duplicate lines from a string.

    Args:
        text (str): Input text with potential duplicate lines

    Returns:
        str: Text with duplicate lines removed
    """
    lines = text.splitlines(keepends=True)
    seen = set()
    unique_lines = []

    for line in lines:
        if line not in seen:
            seen.add(line)
            unique_lines.append(line)

    return ''.join(unique_lines)

def parse_llm_response(response_text):
    """Parse LLM response and extract JSON"""
    try:
        # Try to find JSON in the response
        json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            return json.loads(json_str)
        else:
            # Fallback: try to parse the entire response
            return json.loads(response_text)
    except json.JSONDecodeError:
        return {
            "error": "Failed to parse JSON response",
            "raw_response": response_text
        }

def validate_financial_data(data):
    """Validate extracted financial data"""
    required_fields = ['ticker', 'date', 'shares', 'short_term_debt', 'long_term_debt', 'cash_equivalent']

    for field in required_fields:
        if field not in data:
            return False, f"Missing required field: {field}"

    # Check if values are reasonable (not negative where they shouldn't be)
    numeric_fields = ['shares', 'short_term_debt', 'long_term_debt', 'cash_equivalent']
    for field in numeric_fields:
        if isinstance(data[field], (int, float)) and data[field] < 0:
            return False, f"Negative value for {field}: {data[field]}"

    return True, "Valid"

# Main execution - following your original structure
with open('result.txt', 'w') as f:
    for ticker, startdate in bankrupted[['non_b_ticker', 'startdate']].values:
        company = Company(ticker)
        filings = company.get_filings(form=['10-Q', '10-K'])
        if (len(filings) >= 12):
            try:
                for filing in filings:
                    extracted_text = ''
                    obj = filing.obj()
                    notcompleted = completed[(completed.ticker == ticker) & (completed.report_date == obj.period_of_report)].shape[0] == 0
                    if (obj.period_of_report >= startdate) and (notcompleted):
                        for table in obj.doc.tables():
                            if ('asset' in table.get_text().lower()) or ('share' in table.get_text().lower()) or ('million' in table.get_text().lower()) or ('thousand' in table.get_text().lower()):
                                extracted_text += table.get_text()

                        # Only process if we have relevant data
                        if extracted_text.strip():
                            # # Limit text length to avoid token limits
                            # if len(extracted_text) > 8000:
                            #     extracted_text = extracted_text[:8000] + "\n... [truncated]"
                            text = remove_duplicates_from_string(extracted_text)
                            if len(text) > 80_000:
                                text = text[:80_000]
                            prompt = get_prompt(text, obj.period_of_report, ticker)
                            result = get_result(prompt)

                            # Parse and validate result
                            parsed_result = parse_llm_response(result)

                            if "error" not in parsed_result or "sorry" not in parsed_result:
                                # Write structured output
                                csv_line = f"{parsed_result.get('ticker', ticker)}|{parsed_result.get('date', 'N/A')}|{parsed_result.get('shares', 'N/A')}|{parsed_result.get('short_term_debt', 'N/A')}|{parsed_result.get('long_term_debt', 'N/A')}|{parsed_result.get('cash_equivalent', 'N/A')}|\"{parsed_result.get('notes', '')}\""
                                f.write(csv_line + '\n')
                                print("processed :", ticker, obj.period_of_report)
                            else:
                                # Fallback to original format if parsing fails
                                f.write(result + '\n')
                                print("error processed :", ticker)

                            f.flush()

            except Exception as e:
                print(f"Error processing {ticker}: {e}")
                f.write(f"Error processing {ticker}: {e}\n")
                f.flush()

print("Processing complete!")